In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy import stats

假设以下数据框架：df_stocks（包含股票代码、日期、收益率、因子值、市值、行业）  
示例数据结构：['stock_id', 'date', 'return', 'factor', 'market_value', 'industry']

In [2]:
# 测试数据

np.random.seed(0)  

dates = pd.date_range(start="2020-01-01", end="2020-12-31", freq='D')
data = {
    'stock_id': np.tile(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], len(dates)),
    'date': np.repeat(dates, 10),
    'return': np.random.rand(len(dates) * 10) - 0.5,
    'factor': np.random.rand(len(dates) * 10),
    'market_value': np.random.rand(len(dates) * 10) * 10000,
    'industry': np.tile(['Tech', 'Tech', 'Finance', 'Finance', 'Energy', 'Energy', 'Tech', 'Finance', 'Energy', 'Tech'], len(dates))
}
df = pd.DataFrame(data)
df.head(30)

,stock_id,date,return,factor,market_value,industry
0,A,2020-01-01,0.048814,0.535903,9156.608490,Tech
1,B,2020-01-01,0.215189,0.904397,1255.356503,Tech
2,C,2020-01-01,0.102763,0.294879,4878.892942,Finance
3,D,2020-01-01,0.044883,0.308864,6424.258071,Finance
4,E,2020-01-01,-0.076345,0.627183,5696.041114,Energy
5,F,2020-01-01,0.145894,0.505192,8894.943101,Energy
6,G,2020-01-01,-0.062413,0.195869,6840.334557,Tech
7,H,2020-01-01,0.391773,0.144727,8250.331310,Finance
8,I,2020-01-01,0.463663,0.489497,6621.259107,Energy
9,J,2020-01-01,-0.116558,0.779391,3442.434722,Tech


### 标准化、缩尾极值处理

In [3]:
#scale: 范围乘数
def winsorize(group,scale=5):

    
    # 计算中位数
    median = group['factor'].median()
    # 计算每个值与中位数的绝对差值
    mad = (group['factor'] - median).abs().median()
    
    # 定义去极值的边界
    lower_bound = median - scale * mad
    upper_bound = median + scale * mad
    # 应用去极值的边界
    group['factor'] = group['factor'].apply(lambda x: min(max(x, lower_bound), upper_bound))
    
    return group

In [4]:
def standardize(group):
    # 计算均值和标准差
    mean = group['factor'].mean()
    std = group['factor'].std()
    
#     # 避免除以零的情况，如果标准差为零则返回原值
#     if std == 0:
#         return group
    # 标准化
    group['factor'] = (group['factor'] - mean) / std
    return group

In [5]:
df.groupby('date')['factor'].std()

date
2020-01-01    0.247641
2020-01-02    0.302185
2020-01-03    0.331095
2020-01-04    0.326619
2020-01-05    0.217765
                ...   
2020-12-27    0.251559
2020-12-28    0.342623
2020-12-29    0.320186
2020-12-30    0.234967
2020-12-31    0.233585
Name: factor, Length: 366, dtype: float64

In [6]:
df.groupby('date')['factor'].head()

0       0.535903
1       0.904397
2       0.294879
3       0.308864
4       0.627183
          ...   
3650    0.643393
3651    0.306043
3652    0.864741
3653    0.593604
3654    0.388446
Name: factor, Length: 1830, dtype: float64

In [7]:
df_grouped = df.groupby('date').apply(winsorize)
df_grouped.head(30)




stock_id       date    return    factor  market_value industry
date                                                                        
2020-01-01 0         A 2020-01-01  0.048814  0.535903   9156.608490     Tech
           1         B 2020-01-01  0.215189  0.904397   1255.356503     Tech
           2         C 2020-01-01  0.102763  0.294879   4878.892942  Finance
           3         D 2020-01-01  0.044883  0.308864   6424.258071  Finance
           4         E 2020-01-01 -0.076345  0.627183   5696.041114   Energy
           5         F 2020-01-01  0.145894  0.505192   8894.943101   Energy
           6         G 2020-01-01 -0.062413  0.195869   6840.334557     Tech
           7         H 2020-01-01  0.391773  0.144727   8250.331310  Finance
           8         I 2020-01-01  0.463663  0.489497   6621.259107   Energy
           9         J 2020-01-01 -0.116558  0.779391   3442.434722     Tech
2020-01-02 10        A 2020-01-02  0.291725  0.778831   2725.837965     Tech
           11        B 2020-01-02  0.028895  0.975804   3499.613429     Tech
           12        C 2020-01-02  0.068045  0.382718   4081.781442  Finance
           13        D 2020-01-02  0.425597  0.391989   8268.074735  Finance
           14        E 2020-01-02 -0.428964  0.001819   7109.360508   Energy
           15        F 2020-01-02 -0.412871  0.829917   8268.423644   Energy
           16        G 2020-01-02 -0.479782  0.646628   9251.363961     Tech
           17        H 2020-01-02  0.332620  0.769922   8147.971588  Finance
           18        I 2020-01-02  0.278157  0.470623   9622.775748   Energy
           19        J 2020-01-02  0.370012  0.920193   9498.614396     Tech
2020-01-03 20        A 2020-01-03  0.478618  0.029114   3209.617843     Tech
           21        B 2020-01-03  0.299159  0.115281   3923.070593     Tech
           22        C 2020-01-03 -0.038521  0.376017   5752.057944  Finance
           23        D 2020-01-03  0.280529  0.085106   6905.461963  Finance
           24        E 2020-01-03 -0.381726  0.087888   4647.056708   Energy
           25        F 2020-01-03  0.139921  0.566873   7877.690285   Energy
           26        G 2020-01-03 -0.356647  0.216122   6320.068129     Tech
           27        H 2020-01-03  0.444669  0.626535   5958.702790  Finance
           28        I 2020-01-03  0.021848  0.929400   1453.951190   Energy
           29        J 2020-01-03 -0.085338  0.823882   1713.144307     Tech

In [8]:
df_standardized = df_grouped.reset_index(drop=True).groupby('date').apply(standardize)
df_standardized.head(30)

stock_id       date    return    factor  market_value industry
date                                                                        
2020-01-01 0         A 2020-01-01  0.048814  0.231436   9156.608490     Tech
           1         B 2020-01-01  0.215189  1.719449   1255.356503     Tech
           2         C 2020-01-01  0.102763 -0.741843   4878.892942  Finance
           3         D 2020-01-01  0.044883 -0.685371   6424.258071  Finance
           4         E 2020-01-01 -0.076345  0.600033   5696.041114   Energy
           5         F 2020-01-01  0.145894  0.107418   8894.943101   Energy
           6         G 2020-01-01 -0.062413 -1.141656   6840.334557     Tech
           7         H 2020-01-01  0.391773 -1.348171   8250.331310  Finance
           8         I 2020-01-01  0.463663  0.044043   6621.259107   Energy
           9         J 2020-01-01 -0.116558  1.214662   3442.434722     Tech
2020-01-02 10        A 2020-01-02  0.291725  0.536051   2725.837965     Tech
           11        B 2020-01-02  0.028895  1.187877   3499.613429     Tech
           12        C 2020-01-02  0.068045 -0.774778   4081.781442  Finance
           13        D 2020-01-02  0.425597 -0.744098   8268.074735  Finance
           14        E 2020-01-02 -0.428964 -2.035259   7109.360508   Energy
           15        F 2020-01-02 -0.412871  0.705106   8268.423644   Energy
           16        G 2020-01-02 -0.479782  0.098561   9251.363961     Tech
           17        H 2020-01-02  0.332620  0.506568   8147.971588  Finance
           18        I 2020-01-02  0.278157 -0.483880   9622.775748   Energy
           19        J 2020-01-02  0.370012  1.003851   9498.614396     Tech
2020-01-03 20        A 2020-01-03  0.478618 -1.076753   3209.617843     Tech
           21        B 2020-01-03  0.299159 -0.816504   3923.070593     Tech
           22        C 2020-01-03 -0.038521 -0.029008   5752.057944  Finance
           23        D 2020-01-03  0.280529 -0.907641   6905.461963  Finance
           24        E 2020-01-03 -0.381726 -0.899239   4647.056708   Energy
           25        F 2020-01-03  0.139921  0.547428   7877.690285   Energy
           26        G 2020-01-03 -0.356647 -0.511936   6320.068129     Tech
           27        H 2020-01-03  0.444669  0.727625   5958.702790  Finance
           28        I 2020-01-03  0.021848  1.642360   1453.951190   Energy
           29        J 2020-01-03 -0.085338  1.323668   1713.144307     Tech

### 行业中性和市值中性处理

In [9]:
#输出：中性化后的因子值series
def neutralize(group):
    
    # 取对数市值
    ln_market_value = np.log(group['market_value'])
    # 获取行业哑变量
    industry_dummies = pd.get_dummies(group['industry'])
    # 构建自变量X，包括市值对数和行业哑变量
    X = sm.add_constant(pd.concat([ln_market_value, industry_dummies], axis=1))
    # 回归分析
    model = sm.OLS(group['factor'].astype(float), X.astype(float)).fit()
    # 返回残差
    group['factor_n'] = model.resid
    return group

In [10]:
df_neutralize = df_standardized.reset_index(drop=True).groupby('date').apply(neutralize)
df_neutralize.head(30)

stock_id       date    return    factor  market_value industry  \
date                                                                           
2020-01-01 0         A 2020-01-01  0.048814  0.231436   9156.608490     Tech   
           1         B 2020-01-01  0.215189  1.719449   1255.356503     Tech   
           2         C 2020-01-01  0.102763 -0.741843   4878.892942  Finance   
           3         D 2020-01-01  0.044883 -0.685371   6424.258071  Finance   
           4         E 2020-01-01 -0.076345  0.600033   5696.041114   Energy   
           5         F 2020-01-01  0.145894  0.107418   8894.943101   Energy   
           6         G 2020-01-01 -0.062413 -1.141656   6840.334557     Tech   
           7         H 2020-01-01  0.391773 -1.348171   8250.331310  Finance   
           8         I 2020-01-01  0.463663  0.044043   6621.259107   Energy   
           9         J 2020-01-01 -0.116558  1.214662   3442.434722     Tech   
2020-01-02 10        A 2020-01-02  0.291725  0.536051   2725.837965     Tech   
           11        B 2020-01-02  0.028895  1.187877   3499.613429     Tech   
           12        C 2020-01-02  0.068045 -0.774778   4081.781442  Finance   
           13        D 2020-01-02  0.425597 -0.744098   8268.074735  Finance   
           14        E 2020-01-02 -0.428964 -2.035259   7109.360508   Energy   
           15        F 2020-01-02 -0.412871  0.705106   8268.423644   Energy   
           16        G 2020-01-02 -0.479782  0.098561   9251.363961     Tech   
           17        H 2020-01-02  0.332620  0.506568   8147.971588  Finance   
           18        I 2020-01-02  0.278157 -0.483880   9622.775748   Energy   
           19        J 2020-01-02  0.370012  1.003851   9498.614396     Tech   
2020-01-03 20        A 2020-01-03  0.478618 -1.076753   3209.617843     Tech   
           21        B 2020-01-03  0.299159 -0.816504   3923.070593     Tech   
           22        C 2020-01-03 -0.038521 -0.029008   5752.057944  Finance   
           23        D 2020-01-03  0.280529 -0.907641   6905.461963  Finance   
           24        E 2020-01-03 -0.381726 -0.899239   4647.056708   Energy   
           25        F 2020-01-03  0.139921  0.547428   7877.690285   Energy   
           26        G 2020-01-03 -0.356647 -0.511936   6320.068129     Tech   
           27        H 2020-01-03  0.444669  0.727625   5958.702790  Finance   
           28        I 2020-01-03  0.021848  1.642360   1453.951190   Energy   
           29        J 2020-01-03 -0.085338  1.323668   1713.144307     Tech   

               factor_n  
date                     
2020-01-01 0   0.633065  
           1  -0.093795  
           2  -0.114140  
           3   0.249043  
           4   0.128007  
           5   0.132205  
           6  -1.065102  
           7  -0.134903  
           8  -0.260212  
           9   0.525832  
2020-01-02 10 -0.054477  
           11  0.554706  
           12 -0.357867  
           13 -0.447648  
           14 -1.404769  
           15  1.309821  
           16 -0.700509  
           17  0.805515  
           18  0.094948  
           19  0.200279  
2020-01-03 20 -0.878271  
           21 -0.387555  
           22 -0.042787  
           23 -0.711584  
           24 -1.086716  
           25  0.965963  
           26  0.464530  
           27  0.754371  
           28  0.120753  
           29  0.801296

### 多空组合分组回测

每月rebalance

In [11]:
group_num = 5
interval = 20 #假设20天的数据为1月

In [12]:
# 将日期设为索引，并按月重新采样
df_neutralize.set_index('date', inplace=True)
df_neutralize.head()

,stock_id,return,factor,market_value,industry,factor_n
date,,,,,,
2020-01-01,A,0.048814,0.231436,9156.608490,Tech,0.633065
2020-01-01,B,0.215189,1.719449,1255.356503,Tech,-0.093795
2020-01-01,C,0.102763,-0.741843,4878.892942,Finance,-0.114140
2020-01-01,D,0.044883,-0.685371,6424.258071,Finance,0.249043
2020-01-01,E,-0.076345,0.600033,5696.041114,Energy,0.128007


In [13]:
# 按月末因子值和市值分组, 这里设置20天为1月，进行月度调仓
df_monthly = df_neutralize.groupby('stock_id').resample(str(interval)+'d').first()

In [14]:
df_monthly

stock_id    return    factor  market_value industry  \
stock_id date                                                             
A        2020-01-01        A  0.048814  0.231436   9156.608490     Tech   
         2020-01-21        A -0.188204  0.222321   4284.011695     Tech   
         2020-02-10        A -0.098740 -0.575279   6196.886446     Tech   
         2020-03-01        A -0.325342  0.173809    426.656544     Tech   
         2020-03-21        A -0.460007 -0.610862   9193.473437     Tech   
...                      ...       ...       ...           ...      ...   
J        2020-10-07        J -0.202415  1.344008   8670.986826     Tech   
         2020-10-27        J  0.122927  0.614455   8033.251085     Tech   
         2020-11-16        J -0.225405  0.106524   8443.880346     Tech   
         2020-12-06        J -0.194722 -0.720655   7650.525317     Tech   
         2020-12-26        J  0.066193  1.494853   6377.639241     Tech   

                     factor_n  
stock_id date                  
A        2020-01-01  0.633065  
         2020-01-21  0.050511  
         2020-02-10 -0.500237  
         2020-03-01  0.271094  
         2020-03-21 -0.537282  
...                       ...  
J        2020-10-07  0.953131  
         2020-10-27  0.947205  
         2020-11-16  0.205651  
         2020-12-06 -0.153462  
         2020-12-26  1.024386  

[190 rows x 6 columns]

In [15]:
#按中性化后的因子值排序分组
df_monthly['quantile'] = df_monthly.groupby('date')['factor_n'].transform(lambda x: pd.qcut(x, group_num, labels=False, duplicates='drop'))

In [16]:
df_monthly.head(30)

stock_id    return    factor  market_value industry  \
stock_id date                                                             
A        2020-01-01        A  0.048814  0.231436   9156.608490     Tech   
         2020-01-21        A -0.188204  0.222321   4284.011695     Tech   
         2020-02-10        A -0.098740 -0.575279   6196.886446     Tech   
         2020-03-01        A -0.325342  0.173809    426.656544     Tech   
         2020-03-21        A -0.460007 -0.610862   9193.473437     Tech   
         2020-04-10        A  0.092880 -1.158827   5624.416131     Tech   
         2020-04-30        A -0.191472  0.482469   2829.793171     Tech   
         2020-05-20        A -0.186409  0.581532   6963.224087     Tech   
         2020-06-09        A  0.419171  0.846104   9006.777412     Tech   
         2020-06-29        A  0.046950  0.003937   1459.527948     Tech   
         2020-07-19        A  0.311518  0.957322   9695.786738     Tech   
         2020-08-08        A -0.397928 -1.078203   1821.037975     Tech   
         2020-08-28        A  0.386683  1.420892   7714.059941     Tech   
         2020-09-17        A -0.354052  1.307748   5894.545836     Tech   
         2020-10-07        A -0.065670  1.198852   2987.353972     Tech   
         2020-10-27        A -0.086038  0.100592   2574.445784     Tech   
         2020-11-16        A  0.365805  0.173523   9184.076745     Tech   
         2020-12-06        A -0.004025  0.945308   2999.053572     Tech   
         2020-12-26        A -0.242534  0.745332   4187.363870     Tech   
B        2020-01-01        B  0.215189  1.719449   1255.356503     Tech   
         2020-01-21        B  0.196343 -0.056285   1832.707472     Tech   
         2020-02-10        B  0.429291 -0.225525   2813.264906     Tech   
         2020-03-01        B -0.172012  0.423888   5331.711001     Tech   
         2020-03-21        B  0.139705 -1.491785   2421.201532     Tech   
         2020-04-10        B -0.489936 -1.129132   3612.358113     Tech   
         2020-04-30        B  0.442185  0.596085   3252.251410     Tech   
         2020-05-20        B -0.134461 -0.644949   1122.935362     Tech   
         2020-06-09        B -0.088374  1.442978   9053.849091     Tech   
         2020-06-29        B -0.206383 -1.132321   4652.939637     Tech   
         2020-07-19        B -0.023916  0.773759   2660.792704     Tech   

                     factor_n  quantile  
stock_id date                            
A        2020-01-01  0.633065         4  
         2020-01-21  0.050511         2  
         2020-02-10 -0.500237         1  
         2020-03-01  0.271094         2  
         2020-03-21 -0.537282         1  
         2020-04-10 -0.450440         1  
         2020-04-30 -0.316896         1  
         2020-05-20  0.676783         3  
         2020-06-09 -0.217467         1  
         2020-06-29  0.419389         3  
         2020-07-19  1.189879         4  
         2020-08-08 -1.108304         0  
         2020-08-28  0.457366         3  
         2020-09-17  1.229533         4  
         2020-10-07  0.733980         3  
         2020-10-27  0.785103         4  
         2020-11-16  0.303714         2  
         2020-12-06  1.612502         4  
         2020-12-26  0.348909         3  
B        2020-01-01 -0.093795         2  
         2020-01-21 -0.042740         2  
         2020-02-10 -0.124855         2  
         2020-03-01  0.840708         3  
         2020-03-21 -1.365941         0  
         2020-04-10 -0.050545         1  
         2020-04-30 -0.228758         2  
         2020-05-20  0.431719         3  
         2020-06-09  0.377580         3  
         2020-06-29 -1.218727         0  
         2020-07-19 -0.421457         1

In [17]:
# 筛选最高组和最低组
df_high = df_monthly[df_monthly['quantile'] == group_num - 1]  # 最高组
df_low = df_monthly[df_monthly['quantile'] == 0]  # 最低组

In [18]:
df_high.head(30)

stock_id    return    factor  market_value industry  \
stock_id date                                                             
A        2020-01-01        A  0.048814  0.231436   9156.608490     Tech   
         2020-07-19        A  0.311518  0.957322   9695.786738     Tech   
         2020-09-17        A -0.354052  1.307748   5894.545836     Tech   
         2020-10-27        A -0.086038  0.100592   2574.445784     Tech   
         2020-12-06        A -0.004025  0.945308   2999.053572     Tech   
C        2020-04-10        C -0.024174  1.517690   7536.367849  Finance   
         2020-04-30        C  0.388265  0.473789   4939.689221  Finance   
         2020-05-20        C -0.298733  2.186083   4938.824657  Finance   
D        2020-08-08        D  0.137969  1.645363   3886.714264  Finance   
         2020-10-07        D -0.313696  1.334767   7919.904419  Finance   
         2020-11-16        D  0.152257  1.089959   8352.982416  Finance   
         2020-12-06        D -0.221209  1.040625   9870.146294  Finance   
E        2020-03-01        E  0.107249  0.821137   4675.788186   Energy   
         2020-04-10        E -0.456025  1.429472   7550.405763   Energy   
         2020-12-26        E -0.038188  1.658914   2635.023138   Energy   
F        2020-03-01        F -0.022353  0.805057   3147.689858   Energy   
         2020-03-21        F  0.209035  0.813015   7080.390468   Energy   
         2020-05-20        F  0.412151  0.415953   6140.071031   Energy   
         2020-06-09        F -0.301108  0.422355   7522.174497   Energy   
         2020-06-29        F -0.174145  1.452989   7844.797387   Energy   
G        2020-01-21        G  0.179393  1.361027   7148.299959     Tech   
         2020-03-21        G  0.454334  1.639563   6746.788237     Tech   
         2020-06-09        G  0.156838  1.469654   3599.797702     Tech   
         2020-06-29        G  0.002509  1.560771   6578.946324     Tech   
         2020-08-08        G  0.031568  0.911188   2377.666308     Tech   
         2020-08-28        G -0.476763  1.520844   3559.207293     Tech   
         2020-11-16        G -0.290080  1.363074   4222.991471     Tech   
H        2020-02-10        H -0.267256  1.312839   9539.595102  Finance   
         2020-07-19        H -0.330322  0.522304   7008.001318  Finance   
         2020-08-28        H  0.031104  0.284890   5440.280939  Finance   

                     factor_n  quantile  
stock_id date                            
A        2020-01-01  0.633065         4  
         2020-07-19  1.189879         4  
         2020-09-17  1.229533         4  
         2020-10-27  0.785103         4  
         2020-12-06  1.612502         4  
C        2020-04-10  1.145244         4  
         2020-04-30  0.905085         4  
         2020-05-20  1.196093         4  
D        2020-08-08  1.301648         4  
         2020-10-07  1.131453         4  
         2020-11-16  1.113096         4  
         2020-12-06  0.953355         4  
E        2020-03-01  0.991347         4  
         2020-04-10  0.609701         4  
         2020-12-26  1.870591         4  
F        2020-03-01  0.925198         4  
         2020-03-21  0.776081         4  
         2020-05-20  0.698862         4  
         2020-06-09  0.509764         4  
         2020-06-29  1.252783         4  
G        2020-01-21  1.077453         4  
         2020-03-21  1.725263         4  
         2020-06-09  0.727430         4  
         2020-06-29  1.324432         4  
         2020-08-08  0.779049         4  
         2020-08-28  0.759978         4  
         2020-11-16  1.206049         4  
H        2020-02-10  1.302621         4  
         2020-07-19  0.880903         4  
         2020-08-28  0.804568         4

In [19]:
df_low.head(30)

stock_id    return    factor  market_value industry  \
stock_id date                                                             
A        2020-08-08        A -0.397928 -1.078203   1821.037975     Tech   
B        2020-03-21        B  0.139705 -1.491785   2421.201532     Tech   
         2020-06-29        B -0.206383 -1.132321   4652.939637     Tech   
         2020-09-17        B  0.132766 -1.208906   9196.162961     Tech   
         2020-11-16        B -0.111901 -1.688040   5996.942982     Tech   
         2020-12-06        B  0.475846 -1.748247   4659.537627     Tech   
C        2020-07-19        C  0.023156 -1.225148   7652.020695  Finance   
         2020-08-08        C -0.160349 -0.532045   6394.486915  Finance   
         2020-10-07        C -0.494948 -1.009539   9713.470717  Finance   
         2020-11-16        C  0.295336 -1.383490   5804.126832  Finance   
         2020-12-06        C -0.002367 -0.575686   2920.264475  Finance   
D        2020-02-10        D  0.445302 -1.611170   4332.817492  Finance   
         2020-03-21        D -0.122593 -1.116472   7473.360945  Finance   
         2020-04-30        D  0.360311 -1.468787   7312.004656  Finance   
E        2020-01-21        E -0.475321 -0.575105   5330.356805   Energy   
         2020-08-28        E  0.109161 -1.491328   3890.072991   Energy   
F        2020-04-10        F  0.379521  0.243630   7903.457690   Energy   
         2020-04-30        F -0.155711 -1.244179   4782.173717   Energy   
         2020-07-19        F -0.197095 -1.665880   4276.825940   Energy   
         2020-10-27        F -0.333923 -0.980673   6638.604235   Energy   
G        2020-01-01        G -0.062413 -1.141656   6840.334557     Tech   
         2020-02-10        G -0.173299 -0.973043   6786.354425     Tech   
         2020-03-01        G -0.216000 -1.212563   8813.176806     Tech   
         2020-05-20        G -0.381651 -1.383854   9772.403349     Tech   
         2020-09-17        G  0.216024 -0.973812   5115.861496     Tech   
         2020-10-07        G -0.233495 -0.947315   7100.930299     Tech   
         2020-10-27        G  0.328390 -1.401500   8584.238601     Tech   
         2020-12-26        G  0.279349 -0.808275   2008.246318     Tech   
H        2020-04-10        H -0.469339 -1.067953   8200.676824  Finance   
         2020-05-20        H -0.474810 -0.839540   1410.245276  Finance   

                     factor_n  quantile  
stock_id date                            
A        2020-08-08 -1.108304         0  
B        2020-03-21 -1.365941         0  
         2020-06-29 -1.218727         0  
         2020-09-17 -1.399683         0  
         2020-11-16 -1.715414         0  
         2020-12-06 -1.128104         0  
C        2020-07-19 -0.769473         0  
         2020-08-08 -1.066234         0  
         2020-10-07 -1.198678         0  
         2020-11-16 -1.494936         0  
         2020-12-06 -0.532911         0  
D        2020-02-10 -1.595774         0  
         2020-03-21 -1.250930         0  
         2020-04-30 -1.109306         0  
E        2020-01-21 -1.429113         0  
         2020-08-28 -1.123827         0  
F        2020-04-10 -0.614351         0  
         2020-04-30 -0.821453         0  
         2020-07-19 -0.493445         0  
         2020-10-27 -0.860657         0  
G        2020-01-01 -1.065102         0  
         2020-02-10 -0.900950         0  
         2020-03-01 -0.732155         0  
         2020-05-20 -1.470891         0  
         2020-09-17 -1.016169         0  
         2020-10-07 -1.352063         0  
         2020-10-27 -1.089256         0  
         2020-12-26 -1.075378         0  
H        2020-04-10 -1.511032         0  
         2020-05-20 -1.155407         0

In [20]:
# 市值加权 - 以月末数据计算权重
df_high1 = df_high.copy()
df_low1 = df_low.copy()
df_high1['weight'] = df_high['market_value'] / df_high.groupby('date')['market_value'].transform('sum')
df_low1['weight'] = df_low['market_value'] / df_low.groupby('date')['market_value'].transform('sum')

In [21]:
df_high = df_high1.copy()
df_high.head(30)

stock_id    return    factor  market_value industry  \
stock_id date                                                             
A        2020-01-01        A  0.048814  0.231436   9156.608490     Tech   
         2020-07-19        A  0.311518  0.957322   9695.786738     Tech   
         2020-09-17        A -0.354052  1.307748   5894.545836     Tech   
         2020-10-27        A -0.086038  0.100592   2574.445784     Tech   
         2020-12-06        A -0.004025  0.945308   2999.053572     Tech   
C        2020-04-10        C -0.024174  1.517690   7536.367849  Finance   
         2020-04-30        C  0.388265  0.473789   4939.689221  Finance   
         2020-05-20        C -0.298733  2.186083   4938.824657  Finance   
D        2020-08-08        D  0.137969  1.645363   3886.714264  Finance   
         2020-10-07        D -0.313696  1.334767   7919.904419  Finance   
         2020-11-16        D  0.152257  1.089959   8352.982416  Finance   
         2020-12-06        D -0.221209  1.040625   9870.146294  Finance   
E        2020-03-01        E  0.107249  0.821137   4675.788186   Energy   
         2020-04-10        E -0.456025  1.429472   7550.405763   Energy   
         2020-12-26        E -0.038188  1.658914   2635.023138   Energy   
F        2020-03-01        F -0.022353  0.805057   3147.689858   Energy   
         2020-03-21        F  0.209035  0.813015   7080.390468   Energy   
         2020-05-20        F  0.412151  0.415953   6140.071031   Energy   
         2020-06-09        F -0.301108  0.422355   7522.174497   Energy   
         2020-06-29        F -0.174145  1.452989   7844.797387   Energy   
G        2020-01-21        G  0.179393  1.361027   7148.299959     Tech   
         2020-03-21        G  0.454334  1.639563   6746.788237     Tech   
         2020-06-09        G  0.156838  1.469654   3599.797702     Tech   
         2020-06-29        G  0.002509  1.560771   6578.946324     Tech   
         2020-08-08        G  0.031568  0.911188   2377.666308     Tech   
         2020-08-28        G -0.476763  1.520844   3559.207293     Tech   
         2020-11-16        G -0.290080  1.363074   4222.991471     Tech   
H        2020-02-10        H -0.267256  1.312839   9539.595102  Finance   
         2020-07-19        H -0.330322  0.522304   7008.001318  Finance   
         2020-08-28        H  0.031104  0.284890   5440.280939  Finance   

                     factor_n  quantile    weight  
stock_id date                                      
A        2020-01-01  0.633065         4  0.726770  
         2020-07-19  1.189879         4  0.580454  
         2020-09-17  1.229533         4  0.495794  
         2020-10-27  0.785103         4  0.242696  
         2020-12-06  1.612502         4  0.233041  
C        2020-04-10  1.145244         4  0.499535  
         2020-04-30  0.905085         4  0.655352  
         2020-05-20  1.196093         4  0.445787  
D        2020-08-08  1.301648         4  0.620447  
         2020-10-07  1.131453         4  0.477365  
         2020-11-16  1.113096         4  0.664202  
         2020-12-06  0.953355         4  0.766959  
E        2020-03-01  0.991347         4  0.597661  
         2020-04-10  0.609701         4  0.500465  
         2020-12-26  1.870591         4  0.292369  
F        2020-03-01  0.925198         4  0.402339  
         2020-03-21  0.776081         4  0.512063  
         2020-05-20  0.698862         4  0.554213  
         2020-06-09  0.509764         4  0.676335  
         2020-06-29  1.252783         4  0.543881  
G        2020-01-21  1.077453         4  0.942860  
         2020-03-21  1.725263         4  0.487937  
         2020-06-09  0.727430         4  0.323665  
         2020-06-29  1.324432         4  0.456119  
         2020-08-08  0.779049         4  0.379553  
         2020-08-28  0.759978         4  0.395490  
         2020-11-16  1.206049         4  0.335798  
H        2020-02-10  1.302621         4  0.545708  
         2020-07-19  0.880903         4  0.419546  
       

In [22]:
df_low = df_low1.copy()
df_low.head(30)

stock_id    return    factor  market_value industry  \
stock_id date                                                             
A        2020-08-08        A -0.397928 -1.078203   1821.037975     Tech   
B        2020-03-21        B  0.139705 -1.491785   2421.201532     Tech   
         2020-06-29        B -0.206383 -1.132321   4652.939637     Tech   
         2020-09-17        B  0.132766 -1.208906   9196.162961     Tech   
         2020-11-16        B -0.111901 -1.688040   5996.942982     Tech   
         2020-12-06        B  0.475846 -1.748247   4659.537627     Tech   
C        2020-07-19        C  0.023156 -1.225148   7652.020695  Finance   
         2020-08-08        C -0.160349 -0.532045   6394.486915  Finance   
         2020-10-07        C -0.494948 -1.009539   9713.470717  Finance   
         2020-11-16        C  0.295336 -1.383490   5804.126832  Finance   
         2020-12-06        C -0.002367 -0.575686   2920.264475  Finance   
D        2020-02-10        D  0.445302 -1.611170   4332.817492  Finance   
         2020-03-21        D -0.122593 -1.116472   7473.360945  Finance   
         2020-04-30        D  0.360311 -1.468787   7312.004656  Finance   
E        2020-01-21        E -0.475321 -0.575105   5330.356805   Energy   
         2020-08-28        E  0.109161 -1.491328   3890.072991   Energy   
F        2020-04-10        F  0.379521  0.243630   7903.457690   Energy   
         2020-04-30        F -0.155711 -1.244179   4782.173717   Energy   
         2020-07-19        F -0.197095 -1.665880   4276.825940   Energy   
         2020-10-27        F -0.333923 -0.980673   6638.604235   Energy   
G        2020-01-01        G -0.062413 -1.141656   6840.334557     Tech   
         2020-02-10        G -0.173299 -0.973043   6786.354425     Tech   
         2020-03-01        G -0.216000 -1.212563   8813.176806     Tech   
         2020-05-20        G -0.381651 -1.383854   9772.403349     Tech   
         2020-09-17        G  0.216024 -0.973812   5115.861496     Tech   
         2020-10-07        G -0.233495 -0.947315   7100.930299     Tech   
         2020-10-27        G  0.328390 -1.401500   8584.238601     Tech   
         2020-12-26        G  0.279349 -0.808275   2008.246318     Tech   
H        2020-04-10        H -0.469339 -1.067953   8200.676824  Finance   
         2020-05-20        H -0.474810 -0.839540   1410.245276  Finance   

                     factor_n  quantile    weight  
stock_id date                                      
A        2020-08-08 -1.108304         0  0.221658  
B        2020-03-21 -1.365941         0  0.244700  
         2020-06-29 -1.218727         0  0.322402  
         2020-09-17 -1.399683         0  0.642548  
         2020-11-16 -1.715414         0  0.508169  
         2020-12-06 -1.128104         0  0.614731  
C        2020-07-19 -0.769473         0  0.641472  
         2020-08-08 -1.066234         0  0.778342  
         2020-10-07 -1.198678         0  0.577688  
         2020-11-16 -1.494936         0  0.491831  
         2020-12-06 -0.532911         0  0.385269  
D        2020-02-10 -1.595774         0  0.389671  
         2020-03-21 -1.250930         0  0.755300  
         2020-04-30 -1.109306         0  0.604589  
E        2020-01-21 -1.429113         0  0.837949  
         2020-08-28 -1.123827         0  0.464731  
F        2020-04-10 -0.614351         0  0.490772  
         2020-04-30 -0.821453         0  0.395411  
         2020-07-19 -0.493445         0  0.358528  
         2020-10-27 -0.860657         0  0.436095  
G        2020-01-01 -1.065102         0  0.508137  
         2020-02-10 -0.900950         0  0.610329  
         2020-03-01 -0.732155         0  0.517237  
         2020-05-20 -1.470891         0  0.873890  
         2020-09-17 -1.016169         0  0.357452  
         2020-10-07 -1.352063         0  0.422312  
         2020-10-27 -1.089256         0  0.563905  
         2020-12-26 -1.075378         0  0.230571  
H        2020-04-10 -1.511032         0  0.509228  
       

In [23]:
# 设置用来merge的df
df_t = df_high.drop(['stock_id'],axis=1).reset_index()
df_b = df_low.drop(['stock_id'],axis=1).reset_index()
df_merge = df.reset_index()

In [24]:
df_t.head(30)

,stock_id,date,return,factor,market_value,industry,factor_n,quantile,weight
0,A,2020-01-01,0.048814,0.231436,9156.608490,Tech,0.633065,4,0.726770
1,A,2020-07-19,0.311518,0.957322,9695.786738,Tech,1.189879,4,0.580454
2,A,2020-09-17,-0.354052,1.307748,5894.545836,Tech,1.229533,4,0.495794
3,A,2020-10-27,-0.086038,0.100592,2574.445784,Tech,0.785103,4,0.242696
4,A,2020-12-06,-0.004025,0.945308,2999.053572,Tech,1.612502,4,0.233041
5,C,2020-04-10,-0.024174,1.517690,7536.367849,Finance,1.145244,4,0.499535
6,C,2020-04-30,0.388265,0.473789,4939.689221,Finance,0.905085,4,0.655352
7,C,2020-05-20,-0.298733,2.186083,4938.824657,Finance,1.196093,4,0.445787
8,D,2020-08-08,0.137969,1.645363,3886.714264,Finance,1.301648,4,0.620447
9,D,2020-10-07,-0.313696,1.334767,7919.904419,Finance,1.131453,4,0.477365


In [25]:
df_b.head(30)

,stock_id,date,return,factor,market_value,industry,factor_n,quantile,weight
0,A,2020-08-08,-0.397928,-1.078203,1821.037975,Tech,-1.108304,0,0.221658
1,B,2020-03-21,0.139705,-1.491785,2421.201532,Tech,-1.365941,0,0.244700
2,B,2020-06-29,-0.206383,-1.132321,4652.939637,Tech,-1.218727,0,0.322402
3,B,2020-09-17,0.132766,-1.208906,9196.162961,Tech,-1.399683,0,0.642548
4,B,2020-11-16,-0.111901,-1.688040,5996.942982,Tech,-1.715414,0,0.508169
5,B,2020-12-06,0.475846,-1.748247,4659.537627,Tech,-1.128104,0,0.614731
6,C,2020-07-19,0.023156,-1.225148,7652.020695,Finance,-0.769473,0,0.641472
7,C,2020-08-08,-0.160349,-0.532045,6394.486915,Finance,-1.066234,0,0.778342
8,C,2020-10-07,-0.494948,-1.009539,9713.470717,Finance,-1.198678,0,0.577688
9,C,2020-11-16,0.295336,-1.383490,5804.126832,Finance,-1.494936,0,0.491831


In [26]:
df_daily = pd.merge(df_merge, df_t[['date','stock_id', 'weight']], on=['date', 'stock_id'], how='left')
df_daily.head(30)

,index,stock_id,date,return,factor,market_value,industry,weight
0,0,A,2020-01-01,0.048814,0.535903,9156.608490,Tech,0.72677
1,1,B,2020-01-01,0.215189,0.904397,1255.356503,Tech,NaN
2,2,C,2020-01-01,0.102763,0.294879,4878.892942,Finance,NaN
3,3,D,2020-01-01,0.044883,0.308864,6424.258071,Finance,NaN
4,4,E,2020-01-01,-0.076345,0.627183,5696.041114,Energy,NaN
5,5,F,2020-01-01,0.145894,0.505192,8894.943101,Energy,NaN
6,6,G,2020-01-01,-0.062413,0.195869,6840.334557,Tech,NaN
7,7,H,2020-01-01,0.391773,0.144727,8250.331310,Finance,NaN
8,8,I,2020-01-01,0.463663,0.489497,6621.259107,Energy,NaN
9,9,J,2020-01-01,-0.116558,0.779391,3442.434722,Tech,0.27323


In [27]:
df_daily = pd.merge(df_daily, df_b[['date','stock_id', 'weight']], on=['date', 'stock_id'], how='left')
df_daily.head(30)


,index,stock_id,date,return,factor,market_value,industry,weight_x,weight_y
0,0,A,2020-01-01,0.048814,0.535903,9156.608490,Tech,0.72677,NaN
1,1,B,2020-01-01,0.215189,0.904397,1255.356503,Tech,NaN,NaN
2,2,C,2020-01-01,0.102763,0.294879,4878.892942,Finance,NaN,NaN
3,3,D,2020-01-01,0.044883,0.308864,6424.258071,Finance,NaN,NaN
4,4,E,2020-01-01,-0.076345,0.627183,5696.041114,Energy,NaN,NaN
5,5,F,2020-01-01,0.145894,0.505192,8894.943101,Energy,NaN,NaN
6,6,G,2020-01-01,-0.062413,0.195869,6840.334557,Tech,NaN,0.508137
7,7,H,2020-01-01,0.391773,0.144727,8250.331310,Finance,NaN,NaN
8,8,I,2020-01-01,0.463663,0.489497,6621.259107,Energy,NaN,0.491863
9,9,J,2020-01-01,-0.116558,0.779391,3442.434722,Tech,0.27323,NaN


In [28]:

def fill_weight(group,posit):
    
    indices = group.index[group['weight'+posit].notna()]
    
        
    for idx in indices:
        end_idx = min(idx + 20-1, group.index[-1] + 1)
        group.loc[idx:end_idx, 'weight'+posit] = group.loc[idx, 'weight'+posit]
    return group

In [29]:
df_reset_index = df_daily.groupby('stock_id').apply(lambda x: x.reset_index(drop=True)).reset_index(drop=True)

In [30]:
df_grouped = df_reset_index.groupby('stock_id', group_keys=True).apply(fill_weight,posit='_x')
#df_grouped.head(30)
#a = df_grouped[df_daily['date'] == '2020-01-21']
a = df_grouped[df_grouped['stock_id'] == 'A']
a.head(30)

index stock_id       date    return    factor  market_value  \
stock_id                                                                   
A        0       0        A 2020-01-01  0.048814  0.535903   9156.608490   
         1      10        A 2020-01-02  0.291725  0.778831   2725.837965   
         2      20        A 2020-01-03  0.478618  0.029114   3209.617843   
         3      30        A 2020-01-04 -0.235444  0.248430   3132.872291   
         4      40        A 2020-01-05 -0.140492  0.577297   5107.033625   
         5      50        A 2020-01-06  0.070197  0.533428   6357.439834   
         6      60        A 2020-01-07 -0.341030  0.093384   2370.408602   
         7      70        A 2020-01-08  0.476459  0.179648   5675.669612   
         8      80        A 2020-01-09 -0.182017  0.032673   5631.812410   
         9      90        A 2020-01-10 -0.181431  0.890631   7359.590198   
         10    100        A 2020-01-11  0.177817  0.228618   9667.530779   
         11    110        A 2020-01-12 -0.052875  0.671516   7723.731077   
         12    120        A 2020-01-13  0.225254  0.899943   5558.103014   
         13    130        A 2020-01-14  0.118015  0.861476   7920.308389   
         14    140        A 2020-01-15  0.396547  0.226546   9728.883211   
         15    150        A 2020-01-16 -0.350552  0.691129   8930.837092   
         16    160        A 2020-01-17  0.197429  0.052030   3151.639520   
         17    170        A 2020-01-18  0.021037  0.663457   1695.378496   
         18    180        A 2020-01-19 -0.335306  0.396906   5050.169383   
         19    190        A 2020-01-20 -0.101779  0.211581   9392.329417   
         20    200        A 2020-01-21 -0.188204  0.496307   4284.011695   
         21    210        A 2020-01-22  0.490339  0.583240   5155.929737   
         22    220        A 2020-01-23  0.128982  0.624011   2955.552224   
         23    230        A 2020-01-24 -0.246058  0.063698   5289.325210   
         24    240        A 2020-01-25  0.363856  0.562326    810.007950   
         25    250        A 2020-01-26 -0.144387  0.324786   8227.320657   
         26    260        A 2020-01-27 -0.259171  0.274890    802.051958   
         27    270        A 2020-01-28  0.477495  0.333783    739.897038   
         28    280        A 2020-01-29 -0.206980  0.985155   6080.549795   
         29    290        A 2020-01-30 -0.131415  0.414547   1507.250722   

            industry  weight_x  weight_y  
stock_id                                  
A        0      Tech   0.72677       NaN  
         1      Tech   0.72677       NaN  
         2      Tech   0.72677       NaN  
         3      Tech   0.72677       NaN  
         4      Tech   0.72677       NaN  
         5      Tech   0.72677       NaN  
         6      Tech   0.72677       NaN  
         7      Tech   0.72677       NaN  
         8      Tech   0.72677       NaN  
         9      Tech   0.72677       NaN  
         10     Tech   0.72677       NaN  
         11     Tech   0.72677       NaN  
         12     Tech   0.72677       NaN  
         13     Tech   0.72677       NaN  
         14     Tech   0.72677       NaN  
         15     Tech   0.72677       NaN  
         16     Tech   0.72677       NaN  
         17     Tech   0.72677       NaN  
         18     Tech   0.72677       NaN  
         19     Tech   0.72677       NaN  
         20     Tech       NaN       NaN  
         21     Tech       NaN       NaN  
         22     Tech       NaN       NaN  
         23     Tech       NaN       NaN  
         24     Tech       NaN       NaN  
         25     Tech       NaN       NaN  
         26     Tech       NaN       NaN  
         27     Tech       NaN       NaN  
         28     Tech       NaN       NaN  
         29     Tech       NaN       NaN

In [31]:
df_grouped_low = df_reset_index.groupby('stock_id',group_keys=True).apply(fill_weight,posit='_y')
df_grouped_low.head(30)

index stock_id       date    return    factor  market_value  \
stock_id                                                                   
A        0       0        A 2020-01-01  0.048814  0.535903   9156.608490   
         1      10        A 2020-01-02  0.291725  0.778831   2725.837965   
         2      20        A 2020-01-03  0.478618  0.029114   3209.617843   
         3      30        A 2020-01-04 -0.235444  0.248430   3132.872291   
         4      40        A 2020-01-05 -0.140492  0.577297   5107.033625   
         5      50        A 2020-01-06  0.070197  0.533428   6357.439834   
         6      60        A 2020-01-07 -0.341030  0.093384   2370.408602   
         7      70        A 2020-01-08  0.476459  0.179648   5675.669612   
         8      80        A 2020-01-09 -0.182017  0.032673   5631.812410   
         9      90        A 2020-01-10 -0.181431  0.890631   7359.590198   
         10    100        A 2020-01-11  0.177817  0.228618   9667.530779   
         11    110        A 2020-01-12 -0.052875  0.671516   7723.731077   
         12    120        A 2020-01-13  0.225254  0.899943   5558.103014   
         13    130        A 2020-01-14  0.118015  0.861476   7920.308389   
         14    140        A 2020-01-15  0.396547  0.226546   9728.883211   
         15    150        A 2020-01-16 -0.350552  0.691129   8930.837092   
         16    160        A 2020-01-17  0.197429  0.052030   3151.639520   
         17    170        A 2020-01-18  0.021037  0.663457   1695.378496   
         18    180        A 2020-01-19 -0.335306  0.396906   5050.169383   
         19    190        A 2020-01-20 -0.101779  0.211581   9392.329417   
         20    200        A 2020-01-21 -0.188204  0.496307   4284.011695   
         21    210        A 2020-01-22  0.490339  0.583240   5155.929737   
         22    220        A 2020-01-23  0.128982  0.624011   2955.552224   
         23    230        A 2020-01-24 -0.246058  0.063698   5289.325210   
         24    240        A 2020-01-25  0.363856  0.562326    810.007950   
         25    250        A 2020-01-26 -0.144387  0.324786   8227.320657   
         26    260        A 2020-01-27 -0.259171  0.274890    802.051958   
         27    270        A 2020-01-28  0.477495  0.333783    739.897038   
         28    280        A 2020-01-29 -0.206980  0.985155   6080.549795   
         29    290        A 2020-01-30 -0.131415  0.414547   1507.250722   

            industry  weight_x  weight_y  
stock_id                                  
A        0      Tech   0.72677       NaN  
         1      Tech       NaN       NaN  
         2      Tech       NaN       NaN  
         3      Tech       NaN       NaN  
         4      Tech       NaN       NaN  
         5      Tech       NaN       NaN  
         6      Tech       NaN       NaN  
         7      Tech       NaN       NaN  
         8      Tech       NaN       NaN  
         9      Tech       NaN       NaN  
         10     Tech       NaN       NaN  
         11     Tech       NaN       NaN  
         12     Tech       NaN       NaN  
         13     Tech       NaN       NaN  
         14     Tech       NaN       NaN  
         15     Tech       NaN       NaN  
         16     Tech       NaN       NaN  
         17     Tech       NaN       NaN  
         18     Tech       NaN       NaN  
         19     Tech       NaN       NaN  
         20     Tech       NaN       NaN  
         21     Tech       NaN       NaN  
         22     Tech       NaN       NaN  
         23     Tech       NaN       NaN  
         24     Tech       NaN       NaN  
         25     Tech       NaN       NaN  
         26     Tech       NaN       NaN  
         27     Tech       NaN       NaN  
         28     Tech       NaN       NaN  
         29     Tech       NaN       NaN

In [32]:
df_grouped['weight_y'] = df_grouped_low['weight_y']

In [33]:
df_grouped[['weight_x','weight_y']] = df_grouped[['weight_x','weight_y']].fillna(0)
df_grouped.head(30)

index stock_id       date    return    factor  market_value  \
stock_id                                                                   
A        0       0        A 2020-01-01  0.048814  0.535903   9156.608490   
         1      10        A 2020-01-02  0.291725  0.778831   2725.837965   
         2      20        A 2020-01-03  0.478618  0.029114   3209.617843   
         3      30        A 2020-01-04 -0.235444  0.248430   3132.872291   
         4      40        A 2020-01-05 -0.140492  0.577297   5107.033625   
         5      50        A 2020-01-06  0.070197  0.533428   6357.439834   
         6      60        A 2020-01-07 -0.341030  0.093384   2370.408602   
         7      70        A 2020-01-08  0.476459  0.179648   5675.669612   
         8      80        A 2020-01-09 -0.182017  0.032673   5631.812410   
         9      90        A 2020-01-10 -0.181431  0.890631   7359.590198   
         10    100        A 2020-01-11  0.177817  0.228618   9667.530779   
         11    110        A 2020-01-12 -0.052875  0.671516   7723.731077   
         12    120        A 2020-01-13  0.225254  0.899943   5558.103014   
         13    130        A 2020-01-14  0.118015  0.861476   7920.308389   
         14    140        A 2020-01-15  0.396547  0.226546   9728.883211   
         15    150        A 2020-01-16 -0.350552  0.691129   8930.837092   
         16    160        A 2020-01-17  0.197429  0.052030   3151.639520   
         17    170        A 2020-01-18  0.021037  0.663457   1695.378496   
         18    180        A 2020-01-19 -0.335306  0.396906   5050.169383   
         19    190        A 2020-01-20 -0.101779  0.211581   9392.329417   
         20    200        A 2020-01-21 -0.188204  0.496307   4284.011695   
         21    210        A 2020-01-22  0.490339  0.583240   5155.929737   
         22    220        A 2020-01-23  0.128982  0.624011   2955.552224   
         23    230        A 2020-01-24 -0.246058  0.063698   5289.325210   
         24    240        A 2020-01-25  0.363856  0.562326    810.007950   
         25    250        A 2020-01-26 -0.144387  0.324786   8227.320657   
         26    260        A 2020-01-27 -0.259171  0.274890    802.051958   
         27    270        A 2020-01-28  0.477495  0.333783    739.897038   
         28    280        A 2020-01-29 -0.206980  0.985155   6080.549795   
         29    290        A 2020-01-30 -0.131415  0.414547   1507.250722   

            industry  weight_x  weight_y  
stock_id                                  
A        0      Tech   0.72677       0.0  
         1      Tech   0.72677       0.0  
         2      Tech   0.72677       0.0  
         3      Tech   0.72677       0.0  
         4      Tech   0.72677       0.0  
         5      Tech   0.72677       0.0  
         6      Tech   0.72677       0.0  
         7      Tech   0.72677       0.0  
         8      Tech   0.72677       0.0  
         9      Tech   0.72677       0.0  
         10     Tech   0.72677       0.0  
         11     Tech   0.72677       0.0  
         12     Tech   0.72677       0.0  
         13     Tech   0.72677       0.0  
         14     Tech   0.72677       0.0  
         15     Tech   0.72677       0.0  
         16     Tech   0.72677       0.0  
         17     Tech   0.72677       0.0  
         18     Tech   0.72677       0.0  
         19     Tech   0.72677       0.0  
         20     Tech   0.00000       0.0  
         21     Tech   0.00000       0.0  
         22     Tech   0.00000       0.0  
         23     Tech   0.00000       0.0  
         24     Tech   0.00000       0.0  
         25     Tech   0.00000       0.0  
         26     Tech   0.00000       0.0  
         27     Tech   0.00000       0.0  
         28     Tech   0.00000       0.0  
         29     Tech   0.00000       0.0

In [34]:
# 按日期分组，并计算每个日期的收益
daily_returns = df_grouped.groupby('date').apply(
    lambda x: (x['return'] * (x['weight_x'] - x['weight_y'])).sum()
).reset_index(name='total_return')

# 重命名列
daily_returns.columns = ['date', 'daily_total_return']

# 显示每日总收益序列
daily_returns.head(30)

,date,daily_total_return
0,2020-01-01,-0.192715
1,2020-01-02,0.420095
2,2020-01-03,0.495008
3,2020-01-04,-0.396658
4,2020-01-05,0.098606
5,2020-01-06,-0.080043
6,2020-01-07,-0.687540
7,2020-01-08,0.452736
8,2020-01-09,-0.064156
9,2020-01-10,-0.472944


In [35]:
df_grouped.groupby('date')

### 指标计算

In [36]:
#多空组合的指标

annualized_return = np.mean(daily_returns['daily_total_return']) * 252
annualized_vol = np.std(daily_returns['daily_total_return']) * np.sqrt(252)
sharpe_ratio = annualized_return / annualized_vol

In [37]:
annualized_return

-0.10395290007331347

In [38]:
annualized_vol

4.980086513387699

In [39]:
sharpe_ratio

-0.02087371369831879